In [9]:
!pip install numpy cython


In [13]:
!conda install -c conda-forge scikit-surprise

^C


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate

import warnings
warnings.simplefilter('ignore')

Top N Recommendations

In [17]:
md=pd.read_csv('movies_metadata.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Data in Genre is of the form list of dictionaries, we need to change that.

# # Preprocessing

In [18]:
md['genres']=md['genres'].fillna("[]")

In [19]:
md.head(100)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,NaN,3000000,"[{'id': 18, 'name': 'Drama'}]",NaN,406,tt0113247,fr,La Haine,Aimlessly whiling away their days in the concr...,...,1995-05-31,0.0,98.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,Three Young Friends... One Last Chance.,La Haine,False,7.9,695.0
96,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45549,tt0111173,en,Shopping,"A dark, hip, urban story of a barren and anony...",...,1994-12-06,0.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one leaves without paying...,Shopping,False,5.6,13.0
97,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.nickbroomfield.com/heidifleiss.html,63076,tt0113283,en,Heidi Fleiss: Hollywood Madam,A documentary crew from the BBC arrives in L.A...,...,1995-12-27,0.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Heidi Fleiss: Hollywood Madam,False,6.8,4.0
98,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,11062,tt0115907,en,City Hall,The accidental shooting of a boy in New York l...,...,1996-02-16,0.0,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It started with a shootout on a rainswept stre...,City Hall,False,6.0,67.0


Eval function

In [22]:
list1="[1,2,3,4,5]"
list_eval=eval(list1)
type(list_eval)

list

In [24]:
literal_eval(list1)[0]

1

In [25]:
md['genres']=md['genres'].apply(literal_eval)

In [26]:
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Convert genre to list instead of dict

In [28]:
md['genres']=md['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [30]:
md[md['vote_count'].notnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [34]:
vote_count=md[md['vote_count'].notnull()]['vote_count'].astype('int')

In [35]:
vote_count

0        5415
1        2413
2          92
3          34
4         173
         ... 
45461       1
45462       3
45463       6
45464       0
45465       0
Name: vote_count, Length: 45460, dtype: int32

In [36]:
vote_average=md[md['vote_average'].notnull()]['vote_average'].astype('int')

In [37]:
vote_average

0        7
1        6
2        6
3        6
4        5
        ..
45461    4
45462    9
45463    3
45464    0
45465    0
Name: vote_average, Length: 45460, dtype: int32

In [38]:
top_movies=md.copy()

In [39]:
top_movies1=top_movies.sort_values('vote_average',ascending=False).head(250)

In [40]:
top_movies1

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
21642,False,NaN,0,[Documentary],NaN,320849,tt0886500,en,Ice Age Columbus: Who Were the First Americans?,Firmly rooted in the latest scientific discove...,...,2005-01-01,0.0,0.0,[],Released,NaN,Ice Age Columbus: Who Were the First Americans?,False,10.0,1.0
15710,False,NaN,0,[Documentary],NaN,96451,tt1587373,en,If God Is Willing and da Creek Don't Rise,"In 2006, director Spike Lee created an astonis...",...,2010-08-23,0.0,255.0,[],Released,NaN,If God Is Willing and da Creek Don't Rise,False,10.0,1.0
22396,False,NaN,0,[Documentary],NaN,72123,tt1341746,en,Meat the Truth,Meat the Truth is a high-profile documentary w...,...,2008-10-03,0.0,74.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Meat the Truth,False,10.0,1.0
22395,False,NaN,0,[Documentary],http://www.marvinhamlischmovie.com/,230864,tt3011874,en,Marvin Hamlisch: What He Did For Love,When Marvin Hamlisch passed away in August 201...,...,2013-10-12,0.0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Marvin Hamlisch: What He Did For Love,False,10.0,1.0
35343,False,NaN,300000,"[Comedy, Documentary, Music, TV Movie]",NaN,140595,tt0308213,en,Elaine Stritch: At Liberty,Judy at the Palace. Sinatra at Carnegie Hall. ...,...,2002-01-01,0.0,140.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Legendary performances come along so rarely.,Elaine Stritch: At Liberty,False,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35428,False,NaN,0,"[Drama, Romance]",http://www.kuraitokorode.com/,206155,tt0872007,ja,暗いところで待ち合わせ,"Michiru has lost her eyesight, and to make mat...",...,2006-09-05,0.0,130.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Waiting in the Dark,False,9.0,2.0
24882,False,NaN,0,[],NaN,273334,tt3575800,en,United States of Secrets (Part One): The Program,Last year Edward Snowden downloaded tens of th...,...,2014-05-13,0.0,114.0,[],Released,NaN,United States of Secrets (Part One): The Program,False,9.0,1.0
4183,False,NaN,0,"[Thriller, Drama]",NaN,88727,tt0157411,no,1732 Høtten,A few months after a girl with developmental d...,...,1998-12-26,0.0,100.0,"[{'iso_639_1': 'no', 'name': 'Norsk'}]",Released,NaN,Bloody Angels,False,9.0,2.0
36345,False,NaN,0,"[History, Drama]",NaN,359154,tt4699592,ab,13 dies de octubre,NaN,...,2015-09-10,0.0,0.0,"[{'iso_639_1': 'ca', 'name': 'Català'}, {'iso_...",Released,NaN,13 days of October,False,9.0,1.0


Defining vote threshold

In [41]:
top_movies2=top_movies[top_movies['vote_count']>1000]

In [42]:
top_movies2

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,3.735540e+08,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,2.627972e+08,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,1.874368e+08,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,3.521940e+08,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0
15,False,NaN,52000000,"[Drama, Crime]",NaN,524,tt0112641,en,Casino,The life of the gambling paradise – Las Vegas ...,...,1995-11-22,1.161124e+08,178.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one stays at the top forever.,Casino,False,7.8,1343.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43644,False,NaN,34000000,"[Action, Crime]",NaN,339403,tt3890160,en,Baby Driver,After being coerced into working for a crime b...,...,2017-06-28,2.245113e+08,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,All you need is one killer track.,Baby Driver,False,7.2,2083.0
44009,False,"{'id': 86066, 'name': 'Despicable Me Collectio...",80000000,"[Action, Animation, Adventure, Family, Comedy]",http://www.despicable.me,324852,tt3469046,en,Despicable Me 3,Gru and his wife Lucy must stop former '80s ch...,...,2017-06-15,1.020063e+09,96.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Oh brother.,Despicable Me 3,False,6.2,2002.0
44274,False,"{'id': 173710, 'name': 'Planet of the Apes (Re...",152000000,"[Drama, Science Fiction, War]",http://www.foxmovies.com/movies/war-for-the-pl...,281338,tt3450958,en,War for the Planet of the Apes,Caesar and his apes are forced into a deadly c...,...,2017-07-11,3.699080e+08,140.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,For freedom. For family. For the planet.,War for the Planet of the Apes,False,6.7,1675.0
44678,False,NaN,100000000,"[Action, Drama, History, Thriller, War]",http://www.dunkirkmovie.com/,374720,tt5013056,en,Dunkirk,The miraculous evacuation of Allied soldiers f...,...,2017-07-19,5.198769e+08,107.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The event that shaped our world,Dunkirk,False,7.5,2712.0


In [43]:
top_movies2.sort_values('vote_average',ascending=False).head(250)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
314,False,NaN,25000000,"[Drama, Crime]",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,...,1994-09-23,28341469.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.5,8358.0
40251,False,NaN,0,"[Romance, Animation, Drama]",https://www.funimationfilms.com/movie/yourname/,372058,tt5311514,ja,君の名は。,High schoolers Mitsuha and Taki are complete s...,...,2016-08-26,355298270.0,106.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Your Name.,False,8.5,1030.0
834,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[Drama, Crime]",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,1972-03-14,245066411.0,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0
1152,False,NaN,3000000,[Drama],NaN,510,tt0073486,en,One Flew Over the Cuckoo's Nest,While serving time for insanity at a state men...,...,1975-11-18,108981275.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"If he's crazy, what does that make you?",One Flew Over the Cuckoo's Nest,False,8.3,3001.0
1176,False,"{'id': 119674, 'name': 'Psycho Collection', 'p...",806948,"[Drama, Horror, Thriller]",NaN,539,tt0054215,en,Psycho,When larcenous real estate clerk Marion Crane ...,...,1960-06-16,32000000.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The master of suspense moves his cameras into ...,Psycho,False,8.3,2405.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,False,NaN,76000000,"[Drama, Action, Thriller, Science Fiction]",http://www.universalstudiosentertainment.com/c...,9693,tt0206634,en,Children of Men,"In 2027, in a chaotic world in which humans ca...",...,2006-09-22,69959751.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The future's a thing of the past.,Children of Men,False,7.4,2120.0
24241,False,NaN,58800000,"[War, Action]",http://www.americansnipermovie.com,190859,tt2179136,en,American Sniper,U.S. Navy SEAL Chris Kyle takes his sole missi...,...,2014-12-11,542307423.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The most lethal sniper in U.S. history.,American Sniper,False,7.4,4600.0
13893,False,"{'id': 1241, 'name': 'Harry Potter Collection'...",250000000,"[Adventure, Fantasy, Family]",http://harrypotter.warnerbros.com/harrypottera...,767,tt0417741,en,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...",...,2009-07-07,933959197.0,153.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Dark Secrets Revealed,Harry Potter and the Half-Blood Prince,False,7.4,5435.0
23561,False,NaN,18000000,"[Drama, Comedy]",NaN,194662,tt2562232,en,Birdman,A fading actor best known for his portrayal of...,...,2014-08-27,103215094.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,or (The Unexpected Virtue of Ignorance),Birdman,False,7.4,4657.0


In [46]:
C=vote_average.mean()
C

5.244896612406511

In [48]:
m=vote_count.quantile(0.95)
m

434.0

Getting the year from date

In [54]:
top_movies['year']=pd.to_datetime(top_movies['release_date'],errors='coerce').apply(lambda x: str(x).split('-')[0] if x!=np.nan else np.nan)

In [55]:
top_movies['year'].head(3)

0    1995
1    1995
2    1995
Name: year, dtype: object

In [56]:
top_movies.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


In [60]:
top_movies3=top_movies[(top_movies['vote_count']>=m) & (top_movies['vote_count'].notnull()) & (top_movies['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genres']]
top_movies3['vote_count']=top_movies3['vote_count'].astype('int')
top_movies3['vote_average']=top_movies3['vote_average'].astype('int')
top_movies3.shape
                                                                                                                            

(2274, 6)

In [61]:
def weighted_rating(x):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m) * R)+(m/(m+v) * C)

In [63]:
top_movies3['weight_rate']=top_movies3.apply(weighted_rating,axis=1)

In [64]:
top_movies3=top_movies3.sort_values('weight_rate',ascending=False).head(10)

In [65]:
top_movies3

,title,year,vote_count,vote_average,popularity,genres,weight_ratio,weight_rate
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588,7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871,7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107,7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753,7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787,7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660,7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000,7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927,7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656,7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924,7.851924


In [66]:
top_movies3.drop('weight_ratio',inplace=True,axis=1)

In [67]:
top_movies3

,title,year,vote_count,vote_average,popularity,genres,weight_rate
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


Top movies

In [69]:
#Genre=romance

In [71]:
genre_TM=top_movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1,drop=True)
genre_TM.name='genre'
genre_top_movies=top_movies.drop('genres',axis=1).join(genre_TM)

In [72]:
genre_top_movies

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Action
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Drama
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Thriller
45464,False,NaN,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917,NaN


In [81]:
def build_chart(genre,percentile=0.85):
    df=genre_top_movies[genre_top_movies['genre']==genre]
    vote_counts=df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages=df[df['vote_average'].notnull()]['vote_average'].astype('int')
    c=vote_averages.mean()
    m=vote_counts.quantile(percentile)
    
    qualified=df[(df['vote_count']>=m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genre']]
    qualified['vote_count']=qualified['vote_count'].astype('int')
    qualified['vote_average']=qualified['vote_average'].astype('int')
    
    qualified['wr']=qualified.apply(weighted_rating,axis=1)
    qualified=qualified.sort_values('wr',ascending=False).head(250)
    
    return qualified

In [82]:
build_chart('Animation').head(10)

,title,year,vote_count,vote_average,popularity,genre,wr
359,The Lion King,1994,5520,8,21.6058,Animation,7.799175
5481,Spirited Away,2001,3968,8,41.0489,Animation,7.728370
9698,Howl's Moving Castle,2004,2049,8,16.136,Animation,7.518439
2884,Princess Mononoke,1997,2041,8,17.1667,Animation,7.516883
5833,My Neighbor Totoro,1988,1730,8,13.5073,Animation,7.447452
40251,Your Name.,2016,1030,8,34.461252,Animation,7.183255
5553,Grave of the Fireflies,1988,974,8,0.010902,Animation,7.150771
19901,Paperman,2012,734,8,7.19863,Animation,6.976272
13724,Up,2009,7048,7,19.3309,Animation,6.898194
30315,Inside Out,2015,6737,7,23.9856,Animation,6.893778


In [85]:
build_chart('Family').head(10)

,title,year,vote_count,vote_average,popularity,genre,wr
1225,Back to the Future,1985,6239,8,25.7785,Family,7.820813
359,The Lion King,1994,5520,8,21.6058,Family,7.799175
5481,Spirited Away,2001,3968,8,41.0489,Family,7.728370
5833,My Neighbor Totoro,1988,1730,8,13.5073,Family,7.447452
926,It's a Wonderful Life,1946,1103,8,15.0316,Family,7.222046
19901,Paperman,2012,734,8,7.19863,Family,6.976272
4766,Harry Potter and the Philosopher's Stone,2001,7188,7,38.1872,Family,6.900064
13724,Up,2009,7048,7,19.3309,Family,6.898194
30315,Inside Out,2015,6737,7,23.9856,Family,6.893778
15472,Despicable Me,2010,6595,7,22.2745,Family,6.891633


Content Based recommender

In [88]:
links_small=pd.read_csv('links_small.csv')
links_small=links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')


In [89]:
top_movies=top_movies.drop([19730,29503,35587])

In [90]:
top_movies['id']=top_movies['id'].astype('int')

In [91]:
top_movies4=top_movies[top_movies['id'].isin(links_small)]
top_movies4.shape

(9099, 25)

In [92]:
top_movies4.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


Description and tagline based recommender

In [94]:
top_movies4['tagline']=top_movies4['tagline'].fillna('')
top_movies4['description']=top_movies4['overview']+top_movies4['tagline']
top_movies4['description']=top_movies4['description'].fillna('')


In [95]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix=tf.fit_transform(top_movies4['description'])


In [97]:
tfidf_matrix.shape



(9099, 268124)

Using cosine similarity

In [98]:
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [100]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [101]:
top_movies4=top_movies4.reset_index()
titles=top_movies4['title']
indices=pd.Series(top_movies4.index,index=top_movies4['title'])


In [117]:
def get_recommendation(title):
    idx=indices[title]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=sorted(sim_scores, key=lambda x:x[1],reverse=True)
    sim_scores=sim_scores[0:31]
    movie_indices=[i[0] for i in sim_scores]
    
    return titles.iloc[movie_indices]

In [118]:
get_recommendation('GoldenEye').head(10)

9                         GoldenEye
5172                      Octopussy
2398               Live and Let Die
4309                  Casino Royale
5175          Never Say Never Again
2396             For Your Eyes Only
1883               A View to a Kill
2901    The Man with the Golden Gun
5171            You Only Live Twice
2397                Licence to Kill
Name: title, dtype: object

In [119]:
get_recommendation('Superman').head(10)

2111                            Superman
7718                   All Star Superman
2114    Superman IV: The Quest for Peace
2112                         Superman II
7879           Superman and the Mole-Men
6447                    Superman Returns
8312                        Man of Steel
6693                          Mr. Brooks
5476                        Spider-Man 2
8485                               Enemy
Name: title, dtype: object

In [120]:
get_recommendation('The Apartment').head(10)

727                     The Apartment
1925                Nothing in Common
5242           Shadow of the Thin Man
5769                The Holy Mountain
3904                          48 Hrs.
3221               The House of Mirth
5358                     Safety Last!
1590           The Barefoot Executive
1961                   Running Scared
1003    The Day the Earth Stood Still
Name: title, dtype: object

In [121]:
get_recommendation("Parasite").head(10)

1780                     Parasite
2980                     The Fury
5112                      Spartan
1861           Enemy of the State
4831                Contamination
7054      Futurama: Bender's Game
586             Mrs. Winterbourne
8003                      Lockout
7082                         Igor
8228    Paul Williams Still Alive
Name: title, dtype: object

In [122]:
get_recommendation('The Godfather').head(10)

692               The Godfather
973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
Name: title, dtype: object